In [1]:
!pip install yfinance pandas scikit-learn matplotlib --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
# From LLM https://www.perplexity.ai/search/diligently-crawl-through-to-th-PI2KEBJ8SUyoYVptvwz9Hg
interesting_tickers = [
    "GME",  # GameStop
    "AMC",  # AMC Entertainment
    "PLTR", # Palantir
    "NOK",  # Nokia (ADR)
    "BB",   # BlackBerry
    "F",    # Ford
    "NIO",  # NIO Inc (ADR)
    "BABA", # Alibaba (ADR)
    "PLUG", # Plug Power
    "CCL",  # Carnival Cruise
    "TLRY", # Tilray
    "PFE",  # Pfizer
    "BA",   # Boeing
    "SPCE", # Virgin Galactic
    "WISH", # ContextLogic
    "GLD",  # SPDR Gold Trust
    "UBER", # Uber
    "RKLB", # Rocket Lab
    "RDW",  # Redwire
    "BBAI", # BigBear.ai
    "ASTS", # AST SpaceMobile
    "MVST", # Microvast
    "GEVO", # Gevo
    "ACHR", # Archer Aviation
    "EOSE", # Eos Energy
    "CELH", # Celsius Holdings
    "MARA", # Marathon Digital
    "GSAT", # Globalstar
    "IOVA", # Iovance Biotherapeutics
    "X",    # U.S. Steel
    "ET",   # Energy Transfer
    "DKNG", # DraftKings
    "CLOV", # Clover Health
    "WKHS", # Workhorse Group
    "CLF",  # Cleveland-Cliffs
    "UWMC", # UWM Holdings
    "DIS",  # Walt Disney
    "T",    # AT&T
    "SNAP", # Snap Inc.
    "CGC",  # Canopy Growth
    "RKT",  # Rocket Companies
    "U",    # Unity Software
    "HOOD", # Robinhood
    "SQ",   # Block (Square)
    "BYND", # Beyond Meat
    "COIN", # Coinbase
    "SHOP", # Shopify
    "ZM",   # Zoom
    "FSLY", # Fastly
    "DBX",  # Dropbox
    "M",    # Macy's
    "GPS",  # Gap
    "KO",   # Coca-Cola
    "JNJ",  # Johnson & Johnson
    "WMT",  # Walmart
    "JPM",  # JPMorgan Chase
    "BAC",  # Bank of America
    "C",    # Citigroup
    "GS",   # Goldman Sachs
    "WFC",  # Wells Fargo
    "MO",   # Altria
    "ABBV", # AbbVie
    "CVX",  # Chevron
    "XOM",  # Exxon Mobil
    "BP",   # BP
    "MRK",  # Merck
    "MMM",  # 3M
    "CAT",  # Caterpillar
    "DE",   # Deere & Company
    "RTX",  # Raytheon Technologies
    "LMT",  # Lockheed Martin
    "HON",  # Honeywell
    "GE",   # General Electric
    "CVS",  # CVS Health
    "NKE",  # Nike
    "PG",   # Procter & Gamble
    "V",    # Visa
    "MA",   # Mastercard
    "AXP",  # American Express
    "MCD",  # McDonald's
    "SBUX", # Starbucks
    "HD",   # Home Depot
    "LOW",  # Lowe's
    "TGT",  # Target
    "WBA",  # Walgreens Boots Alliance
    "VZ",   # Verizon
    "INTC", # Intel
    "TWTR", # Twitter (pre-acquisition)
    "SONO", # Sonos
    "SDC",  # SmileDirectClub
    "ACB",  # Aurora Cannabis
    "CGC",  # Canopy Growth
    "SENS", # Senseonics
    "APHA", # Aphria (pre-merger)
    "CRSR", # Corsair Gaming
    "AFRM", # Affirm
    "SI",   # Silvergate Capital
    "CLNE", # Clean Energy Fuels
    "CRWD", # CrowdStrike
]

In [4]:
def is_valid_stock(ticker):
    """
    Check if a stock meets the criteria:
    1. Is listed on the NYSE
    2. Has data for the last 2 years
    
    Args:
        ticker (str): The stock ticker symbol
        
    Returns:
        bool: True if the stock meets both criteria, False otherwise
    """
    try:
        # Get stock info
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Check if listed on NYSE
        if 'exchange' in info:
            exchange = info['exchange']
            if exchange not in ['NYQ', 'NYSE']:
                return False
        else:
            # If exchange info is not available, we can't confirm it's NYSE
            return False
        
        # Get historical data for the past 2 years
        hist = stock.history(period="2y")
        
        # Check if data exists and has sufficient entries
        if hist.empty:
            return False
        
        # Ensure we have at least 400 data points (approx. 80% of trading days in 2 years)
        if len(hist) < 400:
            return False
        
        # Check if the earliest data point is approximately 2 years old
        earliest_date = hist.index[0]
        latest_date = hist.index[-1]
        days_span = (latest_date - earliest_date).days
        
        # We want at least 700 days (approximately 2 years with some flexibility)
        if days_span < 700:
            return False
            
        return True
        
    except Exception as e:
        print(f"Error checking {ticker}: {e}")
        return False

In [5]:
valid_tickers = []
invalid_tickers = []
for ticker in interesting_tickers:
    if is_valid_stock(ticker):
        valid_tickers.append(ticker)
    else:
        invalid_tickers.append(ticker) 

In [6]:
valid_tickers

['GME',
 'AMC',
 'NOK',
 'BB',
 'F',
 'NIO',
 'BABA',
 'CCL',
 'PFE',
 'BA',
 'SPCE',
 'UBER',
 'RDW',
 'BBAI',
 'ACHR',
 'X',
 'ET',
 'CLF',
 'UWMC',
 'DIS',
 'T',
 'SNAP',
 'RKT',
 'U',
 'FSLY',
 'M',
 'KO',
 'JNJ',
 'WMT',
 'JPM',
 'BAC',
 'C',
 'GS',
 'WFC',
 'MO',
 'ABBV',
 'CVX',
 'XOM',
 'BP',
 'MRK',
 'MMM',
 'CAT',
 'DE',
 'RTX',
 'LMT',
 'GE',
 'CVS',
 'NKE',
 'PG',
 'V',
 'MA',
 'AXP',
 'MCD',
 'HD',
 'LOW',
 'TGT',
 'VZ']

In [7]:
def get_stock_name(ticker):
    """
    Get the common name (company name) for a given stock ticker symbol.
    
    Args:
        ticker (str): The stock ticker symbol
        
    Returns:
        str: The company name if found, otherwise the ticker symbol
    """
    try:
        # Create a Ticker object
        stock = yf.Ticker(ticker)
        
        # Get the info dictionary
        info = stock.info
        
        # Return the company name, preferring longName if available
        if 'longName' in info:
            return info['longName']
        elif 'shortName' in info:
            return info['shortName']
        else:
            return ticker  # Return the ticker if no name is found
    
    except Exception as e:
        print(f"Error getting name for {ticker}: {e}")
        return ticker  # Return the ticker symbol in case of error

In [ ]:
tickers = {ticker: get_stock_name(ticker) for ticker in valid_tickers}

In [12]:
res = pd.DataFrame.from_dict(tickers, orient='index', columns=['Company Name']).sort_index()
res.index.name = 'Ticker'
res.reset_index(inplace=True)
res.head()

,Ticker,Company Name
0,ABBV,AbbVie Inc.
1,ACHR,Archer Aviation Inc.
2,AMC,"AMC Entertainment Holdings, Inc."
3,AXP,American Express Company
4,BA,The Boeing Company


In [13]:
res.to_csv('tickers.csv', index=False)